In [2]:
import matplotlib
%matplotlib inline
from pylab import *
import os, h5py
from readsnapsgl import readsnapsgl

matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16

safecolor={ 'silver':'#c0c0c0', 'gray':'#808080', 'black':'#000000', 'red':'#ff0000',
    'maroon':'#800000', 'yellow':'#ffff00', 'olive':'#707030', 'lime':'#00ee00',
    'green':'#009000', 'aqua':'#00eeee', 'teal':'#00a0a0', 'blue':'#0000ff',
    'nave':'#000080', 'fuchsia':'#ff00ff', 'purple':'#900090'}

In [3]:
# The example cluster is from the 300 project run with GIZMO-Simba
# cluster center and R200 from AHF halo catalogue
sf = h5py.File('snap_128.hdf5','r')
cc = np.asarray([  500095.89011853, 500286.04465527,  500481.30095660])
rr = 1446.17

In [4]:
# f = h5py.File(filename, 'w')

# for key in data:
#     f.create_dataset(key, data[key].shape, dtype=data[key].dtype,
#                      compression='gzip')[...] = data[key]

# f.close()

sf.keys()

<KeysViewHDF5 ['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType3', 'PartType4', 'PartType5']>

In [39]:
list(sf['Header'].attrs.items())

[('BoxSize', 1000000.0),
 ('Flag_Cooling', 1),
 ('Flag_DoublePrecision', 0),
 ('Flag_Feedback', 1),
 ('Flag_IC_Info', 3),
 ('Flag_Metals', 11),
 ('Flag_Sfr', 1),
 ('Flag_StellarAge', 1),
 ('HubbleParam', 0.6777),
 ('MassTable', array([0., 0., 0., 0., 0., 0.])),
 ('NumFilesPerSnapshot', 1),
 ('NumPart_ThisFile',
  array([ 2903039,  3198976,   397824, 13902214,   252795,     1770],
        dtype=uint32)),
 ('NumPart_Total',
  array([ 2903039,  3198976,   397824, 13902214,   252795,     1770],
        dtype=uint32)),
 ('NumPart_Total_HighWord', array([0, 0, 0, 0, 0, 0], dtype=uint32)),
 ('Omega0', 0.307115),
 ('OmegaLambda', 0.692885),
 ('Redshift', -2.220446049250313e-16),
 ('Time', 1.0000000000000002)]

In [40]:
fo = h5py.File('snap_CL288.hdf5','w')

In [41]:
dsh=fo.create_group("Header")
for key, value in sf['Header'].attrs.items():
    dsh.attrs[key] = value
# fo.close()

In [42]:
gp0=fo.create_group("PartType0")
gp1=fo.create_group("PartType1")
gp2=fo.create_group("PartType2")
gp3=fo.create_group("PartType3")
gp4=fo.create_group("PartType4")
gp5=fo.create_group("PartType5")

In [29]:
for i in sf["PartType0"].keys():
    print(i)

AGS-Softening
Coordinates
DelayTime
Density
ElectronAbundance
FractionH2
GrackleHI
GrackleHII
GrackleHM
GrackleHeI
GrackleHeII
GrackleHeIII
HaloID
ID_Generations
InternalEnergy
Masses
Metallicity
NWindLaunches
NeutralHydrogenAbundance
ParticleIDs
Potential
Sigma
SmoothingLength
StarFormationRate
Velocities


In [43]:
gpos=sf["PartType0/Coordinates"][:]
idgr=np.sqrt(np.sum((gpos-cc)**2,axis=1))<=rr
Ng0=gpos[idgr,0].size
# save gas particles
for pd in sf["PartType0"].keys():
    gp0.create_dataset(pd, data=sf["PartType0/"+pd][:][idgr])

In [44]:
# dm particles
gpos=sf["PartType1/Coordinates"][:]
idgr=np.sqrt(np.sum((gpos-cc)**2,axis=1))<=rr
Ng1=gpos[idgr,0].size
for pd in sf["PartType1"].keys():
    gp1.create_dataset(pd, data=sf["PartType1/"+pd][:][idgr])

In [45]:
# star particles
gpos=sf["PartType4/Coordinates"][:]
idgr=np.sqrt(np.sum((gpos-cc)**2,axis=1))<=rr
Ng4=gpos[idgr,0].size
for pd in sf["PartType4"].keys():
    gp4.create_dataset(pd, data=sf["PartType4/"+pd][:][idgr])

In [46]:
# bh particles
gpos=sf["PartType5/Coordinates"][:]
idgr=np.sqrt(np.sum((gpos-cc)**2,axis=1))<=rr
Ng5=gpos[idgr,0].size
for pd in sf["PartType5"].keys():
    gp5.create_dataset(pd, data=sf["PartType5/"+pd][:][idgr])

In [48]:
# redo the Header particle numbers
dsh.attrs['NumPart_ThisFile'] = np.array([Ng0,Ng1,0,0,Ng4,Ng5],dtype=np.uint32)
dsh.attrs['NumPart_Total'] = np.array([Ng0,Ng1,0,0,Ng4,Ng5],dtype=np.uint32)
fo.close()